In [1]:
import glob
import xmltodict
import os
from collections import OrderedDict

xml_data = []
itens_limpos = []

for filename in glob.glob('*.xml'):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        data =  xmltodict.parse(f.read())
        xml_data.append(data)
        
com_erro = []
        
for xml in xml_data:
    erro = False   
    itens = xml['CFe']['infCFe']['det']
    if type(itens) == str:
        erro = True
        
    if not erro:   
        for item in itens:
            dados = {
                    'CFe': xml['CFe']['infCFe']['@Id'],
                    'nCFe': xml['CFe']['infCFe']['ide']['nCFe'],
                    'nserieSAT': xml['CFe']['infCFe']['ide']['nserieSAT'],
                    'dEmi': xml['CFe']['infCFe']['ide']['dEmi'],
            } 
            try:
                dados.update(item['prod'])    
                icms = {}
                for key, record in item['imposto']['ICMS'].items():
                    for key, record in record.items():
                        icms['ICMS_' + key] = record
                dados.update(icms)

                pis = {}
                for key, record in item['imposto']['PIS'].items():
                    for key, record in record.items():
                        pis['PIS_' + key] = record
                dados.update(pis)

                cofins = {}
                for record in item['imposto']['COFINS'].values():
                    for key, record in record.items():
                        cofins['COFINS_' + key] = record
                dados.update(cofins)
            except:
                erro = True
            itens_limpos.append(dados)
       
    if erro:
        com_erro.append(xml)

In [3]:
item['imposto']['COFINS']

OrderedDict([('COFINSNT', OrderedDict([('CST', '04')]))])

In [2]:
from pandas import pandas as pd
data_xml = pd.DataFrame(itens_limpos)

In [ ]:
data_xml.to_excel('saida.xlsx')
grouped = data_xml.drop_duplicates(subset=['cProd'], keep='last')
grouped.to_excel('produtos.xlsx')